In [1]:
from scipy.fft import fft

def extract_frequency_domain_features(signal, sampling_rate=4096):
    """
    주파수 도메인 특징 추출
    :param signal: numpy array (1xN 형태)
    :param sampling_rate: 샘플링 주파수 (Hz)
    :return: dict 형태의 주파수 도메인 특징
    """
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1/sampling_rate)[:N//2]
    fft_values = np.abs(fft(signal))[:N//2]
    
    total_power = np.sum(fft_values**2)
    max_frequency = freq[np.argmax(fft_values)]
    mean_frequency = np.sum(freq * fft_values) / np.sum(fft_values)
    median_frequency = freq[np.cumsum(fft_values) >= np.sum(fft_values) / 2][0]
    spectral_skewness = np.mean((freq - mean_frequency)**3 * fft_values) / (np.std(freq) + 1e-10)**3
    spectral_kurtosis = np.mean((freq - mean_frequency)**4 * fft_values) / (np.std(freq) + 1e-10)**4
    peak_amplitude = np.max(fft_values)
    band_energy = np.sum(fft_values[(freq >= 0.1) & (freq <= 1.0)]**2)
    dominant_frequency_power = fft_values[np.argmax(fft_values)]**2
    spectral_entropy = -np.sum((fft_values / np.sum(fft_values)) * np.log2(fft_values / np.sum(fft_values) + 1e-10))
    rms_frequency = np.sqrt(np.mean(fft_values**2))
    variance_frequency = np.var(fft_values)
    
    features = {
        "total_power": total_power,
        "max_frequency": max_frequency,
        "mean_frequency": mean_frequency,
        "median_frequency": median_frequency,
        "spectral_skewness": spectral_skewness,
        "spectral_kurtosis": spectral_kurtosis,
        "peak_amplitude": peak_amplitude,
        "band_energy_0.1_1Hz": band_energy,
        "dominant_frequency_power": dominant_frequency_power,
        "spectral_entropy": spectral_entropy,
        "rms_frequency": rms_frequency,
        "variance_frequency": variance_frequency
    }
    
    return features

import numpy as np

def extract_time_domain_features(signal):
    """
    시간 도메인 특징 추출
    :param signal: numpy array (1xN 형태)
    :return: dict 형태의 시간 도메인 특징
    """
    features = {}
    signal_mean = np.mean(signal)
    signal_std = np.std(signal)
    signal_max = np.max(signal)
    signal_min = np.min(signal)
    signal_rms = np.sqrt(np.mean(signal**2))
    signal_skew = np.mean((signal - signal_mean)**3) / (signal_std**3 + 1e-10)
    signal_kurt = np.mean((signal - signal_mean)**4) / (signal_std**4 + 1e-10)
    signal_peak = np.max(np.abs(signal))
    signal_ppv = signal_max - signal_min  # Peak-to-Peak Value
    signal_crest = signal_peak / (signal_rms + 1e-10)
    signal_impulse = signal_peak / (np.mean(np.abs(signal)) + 1e-10)
    signal_shape = signal_rms / (np.mean(np.abs(signal)) + 1e-10)
    
    features.update({
        "mean": signal_mean,
        "std": signal_std,
        "max": signal_max,
        "min": signal_min,
        "rms": signal_rms,
        "skewness": signal_skew,
        "kurtosis": signal_kurt,
        "peak": signal_peak,
        "ppv": signal_ppv,
        "crest_factor": signal_crest,
        "impulse_factor": signal_impulse,
        "shape_factor": signal_shape
    })
    
    return features

import numpy as np

def compute_fft(signal: np.ndarray, sampling_rate: float):
    """
    입력 신호에 대해 FFT 변환을 수행하고 magnitude, phase, freq를 계산.
    
    :param signal: np.ndarray, 입력 신호 (float32)
    :param sampling_rate: float, 샘플링 레이트 (Hz, float32)
    :return: tuple (magnitude, phase, freq)
        - magnitude: np.ndarray, 진폭 스펙트럼
        - phase: np.ndarray, 위상 스펙트럼
        - freq: np.ndarray, 주파수 축
    """
    # 신호 길이
    n = len(signal)
    
    # FFT 계산
    fft_result = np.fft.fft(signal)
    
    # 주파수 계산
    freq = np.fft.fftfreq(n, d=1 / sampling_rate)
    
    # 진폭 및 위상 계산
    magnitude = np.abs(fft_result)  # FFT의 크기 (magnitude)
    phase = np.angle(fft_result)   # FFT의 위상 (phase)
    
    # 주파수는 대칭이므로 양수 주파수 부분만 반환
    half_n = n // 2
    return magnitude[:half_n], phase[:half_n], freq[:half_n]

# 4. VBL-VA001 Dataset

In [74]:
import pandas as pd
import numpy as np

sampling_rate = 20*1000 # 20kHz
motor_speed = 3000 # 3000 RPM fixed

## 4.1 개별 데이터 확인
- 먼저 개별 데이터(파일)이 어떤 형태로 존재하는 지 확인해 본다.
- 확인 후, 데이터를 센서 단위로 잘라서 np.float32로 바꾸는 작업까지 해둔다.
> 개별 파일에서 x, y, z 를 추출하는 작업을 함수화 


In [68]:
file_path = '/home/lilmae/Desktop/Rotray_Machine_Data/dataset/VBL-VA001/bearing/bearing_000_Ch08_100g_PE_Acceleration.csv'
data_pd = pd.read_csv(file_path, header=None)
data_pd.columns = ['time', 'x', 'y', 'z']

time_np = data_pd['time'].to_numpy(dtype='float32')
x_np = data_pd['x'].to_numpy(dtype='float32')
y_np = data_pd['y'].to_numpy(dtype='float32')
z_np = data_pd['z'].to_numpy(dtype='float32')

print(f'x_len : {len(x_np)}, y_len : {len(y_np)}, z_len : {len(z_np)}')

x_len : 99939, y_len : 99939, z_len : 99939


## 4.2 데이터 interpolation
- 서로 다른 sampling rate 를 가지는 데이터를 활용하기 때문에 이 sampling rate 를 맞추어 주는 것이 필요하다.
- target sampling rate 를 기준으로 데이터를 interpolation 할 수 있도록 한다.

In [69]:
start = time_np[0]
end = time_np[-1]
total = end-start
print(f'start time : {start}, end time : {end}, total_time : {total}')

# 각 기록 시간은 시작시간 기준이 아닌 종료 시간 기준인 것을 알 수 있다. 이를 시작시간 기준으로 바꾸어 interpolation을 용이하게 해준다.
time_np = time_np - start

start = time_np[0]
end = time_np[-1]
total = end-start
print(f'start time : {start}, end time : {end}, total_time : {total}')

start time : 0.00292899995110929, end time : 4.999981880187988, total_time : 4.9970526695251465
start time : 0.0, end time : 4.9970526695251465, total_time : 4.9970526695251465


In [76]:
from scipy.interpolate import interp1d

signal = x_np # 센서 하나씩 interpolation할 수 있도록 한다 : 예시 x-sensor
target_sampling = 10*1000 # 기준으로 정한 sampling rate : 예시 10kHz
target_time = np.linspace(0, end, target_sampling, endpoint=False) # target sampling rate에 맞도록 시간축 생성

interpolator = interp1d(time_np, signal, kind='linear')
interpolated_signal = interpolator(target_time)

## 4.3 전체 데이터를 순회해서 접근
- 전체 데이터를 순회하여 접근하고 데이터 파일과 클래스 정보를 확인하는 작업을 해준다.

In [89]:
import os

vbl_root = os.path.join(os.getcwd(), 'dataset', 'VBL-VA001')

for class_name in os.listdir(vbl_root):
    class_dir = os.path.join(vbl_root, class_name)
    
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)

    

/home/lilmae/Desktop/Rotray_Machine_Data/dataset/VBL-VA001/normal/normal_125-2_Ch08_100g_PE_Acceleration.csv
